In [147]:
import numpy as np
import base64
from PIL import Image
from io import BytesIO

def generate(src):
    globals()["src"] = src
    arr = np.array(Image.open(BytesIO(base64.b64decode(src.split(",")[-1]))))
    arr = 255 - arr[:,:,:3]
    byte_io = BytesIO()
    Image.fromarray(arr).save(byte_io, 'PNG')
    txt = b'data:image/png;base64,'+base64.b64encode(byte_io.getvalue())
    return txt

from IPython.display import HTML
HTML("./main.html")

In [110]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.imshow(arr)